# silver.ipynb
Applies cleaning and transformation to the raw data in the bronze layer

In [1]:
from pyspark.sql import SparkSession
from spark_config import configure_spark_session

In [2]:
# Stop any existing spark sessions, from previous jupyter runs
spark = SparkSession.builder.getOrCreate()
spark.stop()

# Create a new spark session
builder = SparkSession.builder
builder.appName("silver")
builder.master("spark://spark-master:7077")

# Apply common spark configs, for the bronze catalog and access to minio
configure_spark_session(builder)

spark = builder.getOrCreate()

In [3]:
spark.sql("""
    CREATE SCHEMA IF NOT EXISTS example.silver
    LOCATION 's3a://silver'
""")

DataFrame[]

In [5]:
spark.sql("""
CREATE TABLE IF NOT EXISTS example.silver.page_load (
    event_name STRING,
    event_version STRING,
    event_ts TIMESTAMP,
    page STRING,
    user_name STRING,
    browser STRING
)
USING ICEBERG
""")

DataFrame[]

In [6]:
# Rename and typecast bronze columns into the silver schema
df = spark.sql(f"""
SELECT
    metadata.name AS event_name,
    metadata.version AS event_version,
    CAST(metadata.timestamp AS TIMESTAMP) AS event_ts,
    payload.page AS page,
    payload.user_name AS user_name,
    payload.browser AS browser
FROM example.bronze.page_load_v1
ORDER BY event_ts
""")
df.show()
print(f"Number of rows {df.count()}")

+----------+-------------+--------------------+---------+------------------+-------+
|event_name|event_version|            event_ts|     page|         user_name|browser|
+----------+-------------+--------------------+---------+------------------+-------+
| page_load|           v1|2025-03-27 00:59:...|    /home|    Joseph Fischer|   Edge|
| page_load|           v1|2025-03-27 01:16:...|    /home|    Joseph Fischer|   Edge|
| page_load|           v1|2025-03-27 01:23:...|    /home|       Jason Hayes|Firefox|
| page_load|           v1|2025-03-27 01:28:...| /contact|        Joan Scott|Firefox|
| page_load|           v1|2025-03-27 01:45:...|    /home|        Joan Scott| Chrome|
| page_load|           v1|2025-03-27 01:49:...|    /cart|  Alexandra Morgan| Safari|
| page_load|           v1|2025-03-27 02:19:...|/checkout|Patricia Patterson|   Edge|
| page_load|           v1|2025-03-27 02:30:...|/checkout|    Joseph Fischer|Firefox|
| page_load|           v1|2025-03-27 02:47:...|/checkout|     Dam

In [7]:
# Remove any duplicate rows, because bronze jobs can cause duplication if they process the same file twice
df = df.dropDuplicates()
df.show()
print(f"Number of rows {df.count()}")

+----------+-------------+--------------------+---------+------------------+-------+
|event_name|event_version|            event_ts|     page|         user_name|browser|
+----------+-------------+--------------------+---------+------------------+-------+
| page_load|           v1|2025-03-31 01:45:...|   /about|     Cynthia Brock|Firefox|
| page_load|           v1|2025-03-29 10:09:...|/products|       Lee Hawkins|Firefox|
| page_load|           v1|2025-03-31 14:31:...|/checkout|     Damon Bennett| Chrome|
| page_load|           v1|2025-03-30 08:02:...|    /home|     Cynthia Brock|Firefox|
| page_load|           v1|2025-04-01 16:23:...|/products|     Cynthia Brock|   Edge|
| page_load|           v1|2025-03-31 17:48:...|    /home|      Chad Nichols|   Edge|
| page_load|           v1|2025-03-31 21:23:...|    /home|     Calvin Murray|Firefox|
| page_load|           v1|2025-04-01 03:23:...| /contact|        Joan Scott|   Edge|
| page_load|           v1|2025-04-02 06:50:...|    /home|       J

In [9]:
# Write the output to the silver table
df.write \
    .format("iceberg") \
    .mode("overwrite") \
    .saveAsTable("example.silver.page_load")
print('Done')

Done


In [10]:
spark.stop()